<a href="https://colab.research.google.com/github/hihikaili/Genomic/blob/master/GenomicDataScienceCapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ps -aux|grep python

In [0]:
!kill -9 2157 2217 2285 2287

In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [0]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [0]:
!git clone https://github.com/alexdobin/STAR.git

Upload Human genome data (fasta file) and annotation (GTF or GFF file)
https://useast.ensembl.org/info/data/ftp/index.html

Use the soft-maksed primary assembly file, see explanation:
https://bioinformatics.stackexchange.com/questions/540/what-ensembl-genome-version-should-i-use-for-alignments-e-g-toplevel-fa-vs-p

In [0]:
!mkdir HumanGenome
!gunzip -c /content/Homo_sapiens.GRCh38.100.gtf.gz > /content/HumanGenome/GRCh38.gtf
!gunzip -c /content/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa.gz > /content/HumanGenome/GRCh38.fasta
!head /content/HumanGenome/GRCh38.fasta
!head /content/HumanGenome/GRCh38.gtf

In [0]:
!/content/STAR/bin/Linux_x86_64/STAR --runThreadN 32 --runMode genomeGenerate --genomeDir /content/HumanGenome --genomeFastaFiles /content/HumanGenome/GRCh38.fasta --sjdbGTFfile /content/HumanGenome/GRCh38.gtf --sjdbOverhang 100 --genomeSAsparseD 2 --genomeSAindexNbases 14

Download the output genome index

Download the sequence need to be aligned (SRA files) from NCBI, e.g.:
https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR1554537

Use the SRA toolkit to convert the SRA files into fastq files or directly download the fastq files using fastq-dump or fasterq-dump, see:
https://reneshbedre.github.io/blog/fqutil.html


In [0]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

!conda config --add channels defaults
!conda config --add channels bioconda
!conda config --add channels conda-forge

In [0]:
!conda install -y sra-tools

Connect to Hard Drive for large data storage by using Jupyter server, see:
https://github.com/googlecolab/jupyter_http_over_ws

If the above is not working, the most efficient way is downloading the data from European Nucleotide Archive (https://www.ebi.ac.uk/) FTP using python, and use the gz file in mapping, see the following:

In [0]:
import shutil
import urllib.request as request
from contextlib import closing

with closing(request.urlopen('ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR207/007/SRR2071377/SRR2071377_2.fastq.gz')) as r:
    with open('SRR2071377_2.fastq.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [0]:
!cat SRR1554566* > all_SRR1554566.fastq.gz

In [0]:
!cat SRR2071377* > all_SRR2071377.fastq.gz

In [0]:
!/content/STAR/bin/Linux_x86_64/STAR --runThreadN 32 --genomeDir /content/HumanGenome --readFilesIn /content/all_SRR1554566.fastq.gz /content/all_SRR2071377.fastq.gz --readFilesCommand gunzip -c --outFileNamePrefix SRX683824_ --quantMode GeneCounts --outFilterScoreMinOverLread 0.3 --outFilterMatchNminOverLread 0.3 --outSAMtype BAM Unsorted

In [0]:
!split -b 1000000000 /content/SRX683824_Aligned.out.bam SRX683824_Aligned.out.bam

In [0]:
!head -30 /content/SRX683824_Log.final.out
!head /content/SRX683824_SJ.out.tab
!head /content/SRX683824_ReadsPerGene.out.tab